In [184]:
from hello.pid.lvpid import PIDController
from hello.pid.delay import seconds, minutes, hours, days, m2s, s2m, h2s, DelaySink, DelayBuffer
import numpy as np

from hello.pid.gas_process import GasController, HeadspaceProcess

In [199]:
# Henry's law
C = 2400 
T = 273.15 + 37
T0 = 298
hcp0 = 3.4e-2
hcp = hcp0*np.exp(C*(1/T - 1/T0))
hcp

0.034000000000000002

In [224]:
TOTAL_RVOLUME = {
    3: 4,
    15: 17,
    80: 90,
}

def qf(a,b,c):
    return (-b+np.sqrt(b**2-4*a*c))/(2*a)

def ph_to_co2(ph, hco3):
    h = 10**(-ph)
    w = 1e-7
    ka = 4.45e-7
    bc = hco3 + h - w
    co2 = bc * h / ka
    return co2 / hcp

def general_ph(c1, c2, c3, ka):
    a = 1
    b = c2+c3+ka
    c = c2*c3-ka*c1
    x=qf(a,b,c)
    return -np.log10(c2+x)

def cpH(co2, bicarb):
    # co2 as fraction of atmosphere
    c1 = co2 * hcp # henry's law
    c2 = 1e-7
    c3 = bicarb
    ka = 4.45e-7  # pka 6.35
    a = 1
    b = c2+c3+ka
    c = c2*c3-ka*c1
    x=qf(a,b,c)
    return -np.log10(c2+x)

class pHProcess():
    """ pH Process
    Uses cascaded calculation model,
    First using HeadspaceProcess to estimate
    headspace gas concentrations, and using
    the result to calculate change in PV. 
    """
    
    # First order constant
    default_k = (-0.19053+-0.17422)/2
    
    # Consumption rate
    default_c = 0
    
    # volume fraction of CO2, N2, O2 in air
    # note the remaining 1% is trace gasses
    AIR_CNO = (0.0004, 0.7809, 0.2095)
    
    def __init__(self, main_gas=1, initial_pv=7, initial_cno=AIR_CNO, reactor_size=80, volume=55, delay=0,
                 bicarb=2.02):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        :param bicarb: bicarbonate concentration in g/L
        """
        self.sink = DelaySink(delay, initial_cno[2])
        self.delay = self.sink.delay
        
        self.k = 0
        self.k_mult = 0
        self._k = 0
        self.c = 0
        self.dc = 0
        self.d2c = 0
        self.set_values(k=self.default_k, k_mult=1, c=self.default_c, dc=0, d2c=0)
        self._bicarb = bicarb / 84  # moles / L
        self._co2 = ph_to_co2(initial_pv, self._bicarb)
        
        self._reactor_size = reactor_size
        self._volume = volume
        self.main_gas = main_gas
    
        hs_volume = TOTAL_RVOLUME[reactor_size] - volume
        c, n, o = initial_cno
        self.hp = HeadspaceProcess(hs_volume, c, n, o)
        
    @property
    def volume(self): 
        return self._volume
    
    def set_values(self, **kw):
        if any(not hasattr(self, a) for a in kw):
            raise KeyError(next(set(kw) - set(dir(self))))
            
        if kw.get('k') is not None or kw.get('k_mult') is not None:
            if kw.get('k') is not None:
                self.k = kw['k'] / 3600
            
            if kw.get('k_mult') is not None:
                self.k_mult = kw['k_mult']
                
            self._k = self.k_mult * self.k
    
        if kw.get('c') is not None:
            self.c = kw['c'] / 3600
        if kw.get('dc') is not None:
            self.dc = kw['dc'] / 3600 / 3600
        if kw.get('d2c') is not None:
            self.d2c = kw['d2c'] / 3600 / 3600 / 3600
            
        if kw.get('delay') is not None:
            self.sink.set_delay(kw['delay'])
    
    @volume.setter
    def volume(self, v):
        self._volume = v
        self.hp.vol = TOTAL_RVOLUME[self._reactor_size] - v

    def step(self, co2_req, n2_req, o2_req, air_req):
        
        # The convention of having pv as a parameter to the step 
        # function to minimize the class statefulness has to be 
        # broken here to track the concentration of [CO2] accurately. 
        # Because pH calculations are approximations using equilibrium
        # equations and involve very small numbers, small errors can
        # propagate to significant errors over time. 
        
        # [CO2] is tracked internally as the expected atmospheric 
        # concentration for the system's current PV. PV itself is not
        # stored internally - it is calculated on demand based on 
        # [CO2], assuming the system is at equilibrium (i.e., that
        # the equilibrium reaction happens much faster than [CO2]
        # changes).
        
        # This appears to corroborate with data collected on 3/14/17.
        
        self.hp.calc_gas(self.main_gas, co2_req, n2_req, o2_req, air_req)
        co2_hs = self.hp.co2A
        co2 = self._co2
        dco2 = self._k * (co2 - co2_hs)
        co2 += dco2
        
        self.dc += self.d2c
        self.c += self.dc
        co2 += self.c
        
        
        if co2 < 0:
            co2 = 0
        self._co2 = co2
        pv = cpH(co2, self._bicarb)
        return pv 

In [225]:
cfg.simops = ops
ops.bicarb = 2.02
run_sim()

Changed value: c to -1.0
Changed value: c to -0.01
Changed value: c to -0.01
State Value Updated: sp 8.0 
State Value Updated: sp 87.0 
Changed value: c to 0.0
Changed value: c to 0.0
State Value Updated: sp 7.21 
Value Changed: co2_pid.pgain: -100.00
Value Changed: co2_pid.pgain: 100.00
Value Changed: co2_pid.pgain: -100.00
Value Changed: co2_pid.pgain: -300.00
Value Changed: co2_pid.pgain: -200.00
Advancing simulation 36000 seconds...
Advancing simulation 36000 seconds...
Advancing simulation 36000 seconds...
Advancing simulation 36000 seconds...
Advancing simulation 36000 seconds...
Advancing simulation 36000 seconds...
Advancing simulation 36000 seconds...
Advancing simulation 36000 seconds...
Advancing simulation 36000 seconds...
Advancing simulation 36000 seconds...
Advancing simulation 36000 seconds...
Advancing simulation 36000 seconds...
Advancing simulation 36000 seconds...
Advancing simulation 36000 seconds...
Advancing simulation 36000 seconds...
Advancing simulation 36000 

In [219]:
import json
from pysrc.snippets import OptionCategory

class PIDOps(OptionCategory):
    p = 5
    i = 5
    d = 0
    amax = 100
    amin = 0
    alpha = 1
    beta = 1
    linearity = 1
    gamma = 0
    deadband = 0
    man_request = 0
    mode = 0
    
class MFCOps(OptionCategory):
    co2_max = 1
    o2_max = 2
    n2_max = 10
    air_max = 10
    
class PlotOps(OptionCategory):
    xscale = 'auto'
    xmin = 0
    xmax = 72
    xscale_factor = 3600

class SimOps(OptionCategory):
    
    co2_pid = PIDOps()
    co2_pid.p = -200
    co2_pid.i = 2
    co2_pid.d = 0
    co2_pid.amax = 100
    co2_pid.amin = 0
    co2_pid.beta = 1
    co2_pid.linearity = 1
    co2_pid.alpha = 1
    
    base_pid = PIDOps()
    base_pid.p = 20
    base_pid.i = 2

    mfcs = MFCOps()
    mfcs.co2_max = 1
    mfcs.o2_max = 10
    mfcs.n2_max = 10
    mfcs.air_max = 10
    
    plots = PlotOps()
    plots.xscale = 'auto'
    plots.xmin = 0
    plots.xmax = 72
    plots.xscale_factor = 3600
    
    delay = 0
    end = 10000
    initial_actual_cno = pHProcess.AIR_CNO
    initial_request_cno = (0.07, 0, 0)
    initial_request_base = 0
    initial_pv = 90
    set_point = 40
    set_point_deadband = 1
    k_mult = 1.1
    k = None
    c = None
    dc = 0
    d2c = 0
    mode = "m2a"
    main_gas = 1.0
    reactor_size = 80
    reactor_volume = reactor_size * 55/80
    time_unit = hours
    max_iters = 3 * days
    bicarb = 3.7


In [220]:
ops = SimOps()

ops.co2_pid.p = -200
ops.co2_pid.i = 2
ops.co2_pid.d = 0
ops.co2_pid.amax = 30
ops.co2_pid.amin = 0
ops.co2_pid.alpha = -1
ops.co2_pid.linearity = 1
ops.co2_pid.beta = 0
ops.co2_pid.gamma = 0
ops.co2_pid.deadband = 0
ops.co2_pid.man_request = 0
ops.co2_pid.mode = 0

ops.base_pid.p = 20
ops.base_pid.i = 2
ops.base_pid.d = 0
ops.base_pid.amax = 20
ops.base_pid.amin = 0
ops.base_pid.alpha = -1
ops.base_pid.beta = 1
ops.base_pid.linearity = 1
ops.base_pid.gamma = 0
ops.base_pid.deadband = 0
ops.base_pid.man_request = 0
ops.base_pid.mode = 0

ops.mfcs.co2_max = 1
ops.mfcs.o2_max = 2
ops.mfcs.n2_max = 10
ops.mfcs.air_max = 10

ops.delay = 0
ops.end = 20*hours
ops.initial_actual_cno = pHProcess.AIR_CNO
ops.initial_request_cno = (0.07, 0, 0)
ops.initial_pv = 7
ops.set_point = 7.21
ops.set_point_deadband = 0.02
ops.k = pHProcess.default_k
ops.k_mult = 1.0
ops.c = 0
ops.dc = 0
ops.d2c = 0
ops.mode = "o2a"
ops.main_gas = 2.0
ops.reactor_size = 80
ops.reactor_volume = 55
ops.time_unit = hours
ops.max_iters = 7 * days
ops.bicarb = 3.7

In [211]:
def _setpid(ob, state, name, value):
    pid = state[ob]
    if name == "pgain":
        pid.pgain = value
    elif name == "itime":
        pid.itime = m2s(value)
    elif name == "dtime":
        pid.dtime = m2s(value)
    elif name == "beta":
        pid.b = value
    elif name == "mode":
        sp = state['sp']
        if ob[:2] == "n2":
            sp += state['db']
        elif ob[:2] == 'o2':
            sp -= state['db']
        else:
            print("BAD PID")
        pid.set_mode(value, state['pv'], sp)
    elif name == "man":
        pid.man_request = int(value)
    else:
        print("WARNING: Invalid attribute for %r: %r" % (ob, name, name == "man"))

def _update_value(ops, state, proc, ob, name, value):
    if ob == "process":
        if name == "delay":
            value *= 60
        proc.set_values(**{name:value})
        return
    elif ob in ("co2_pid", "base_pid"):
        _setpid(ob, state, name, value)
        return
    print("WARNING: Invalid attribute or object: %r %r" % (ob, name))


def ph_sim_coroutine(ops, state, xq, pvq, cq,
                     co2q, n2q, o2q, aq, baseq,
                     co2ukq, co2upq, co2uiq, co2udq,
                     baseukq, baseupq, baseuiq, baseudq):
    
    """ This is the longest function in the history of man. 
    Its long becaues there are a *lot* of values to use and unpack for the inner PID loop,
    including code needed to synchronize with UI updates.
    Yay. 
    """
    
    co2_pid = PIDController(pgain=ops.co2_pid.p,              itime=m2s(ops.co2_pid.i),
                            dtime=m2s(ops.co2_pid.d),         auto_max=ops.co2_pid.amax,
                            auto_min=ops.co2_pid.amin,       
                            beta=ops.co2_pid.beta,
                            linearity=ops.co2_pid.linearity,  alpha=ops.co2_pid.alpha,
                            deadband=ops.co2_pid.deadband,    sp_high=100, sp_low=0,
                            gamma=ops.co2_pid.gamma,          man_request=ops.co2_pid.man_request,
                            mode=ops.co2_pid.mode)
    
    base_pid = PIDController(pgain=ops.base_pid.p,              itime=m2s(ops.base_pid.i),
                            dtime=m2s(ops.base_pid.d),         auto_max=ops.base_pid.amax,
                            auto_min=ops.base_pid.amin,       
                            beta=ops.base_pid.beta,
                            linearity=ops.base_pid.linearity,  alpha=ops.base_pid.alpha,
                            deadband=ops.base_pid.deadband,    sp_high=100, sp_low=0,
                            gamma=ops.base_pid.gamma,          man_request=ops.base_pid.man_request,
                            mode=ops.base_pid.mode)

    co2_req, n2_req, o2_req = ops.initial_request_cno
    base_req = ops.initial_request_base
    air_req = max(1-co2_req - n2_req - o2_req, 0)
    pv = ops.initial_pv
    sp = ops.set_point
    mode = ops.mode
    db = ops.set_point_deadband
    
    if mode == "o2a":
        co2_req = 0
        base_req = 0
        co2_pid.off_to_auto(pv, sp+db)
        base_pid.off_to_auto(pv, sp-db)

    if mode == 'm2a':
        co2_pid.man_to_auto(pv, sp+db, co2_req*100)
        base_pid.man_to_auto(pv, sp-db, base_req*100)
        
    delay = ops.delay
    mg = ops.main_gas
    co2a, n2a, o2a = ops.initial_actual_cno
    
    proc = pHProcess(mg, pv,     (co2a, n2a, o2a), 
                     ops.reactor_size, ops.reactor_volume, 
                     ops.delay, ops.bicarb)
    
    proc.set_values(k=ops.k, k_mult=ops.k_mult, c=ops.c, dc=ops.dc, d2c=ops.d2c)
    ctrl = GasController(ops.mfcs.co2_max, ops.mfcs.n2_max, ops.mfcs.o2_max, ops.mfcs.air_max)
    
    t = 0
    time_unit = ops.time_unit
    mi = ops.max_iters
    msg = None

    while True:

        cmd, arg = yield msg
        msg = None
        
        state['c'] = proc.c * 3600
        state['dc'] = proc.dc * 3600 * 3600
        state['d2c'] = proc.d2c * 3600 * 3600 * 3600
        state['k'] = proc.k
        state['sp'] = sp
        state['db'] = db
        state['pv'] = pv
        state['mg'] = mg
        state['t'] = t
        state['co2_req'] = co2_req
        state['n2_req'] = n2_req
        state['o2_req'] = o2_req
        state['air_req'] = air_req
        state['co2_pid'] = co2_pid
        state['base_pid'] = base_pid
        state['base_req'] = base_req
        state['ph_process'] = proc

        if cmd == "SIM_ITERS":
            
            iters = arg
            if iters < 0:
                continue
            elif iters > mi:
                iters = mi

            # One approximation made in this simulation
            # is that no values, including oxygen consumption
            # (c), change during the inner iteration period 
            # outside of modification by the process or PID
            # classes themselves. 

            while iters > 0:

                t += 1
                iters -= 1
                co2_req = co2_pid.step(pv, sp+db) / 100
                base_req = base_pid.step(pv, sp-db) / 100
                co2_req, n2_req, o2_req, air_req = ctrl.request(mg, co2_req, n2_req, o2_req)
                pv = proc.step(co2_req, n2_req, o2_req, air_req)

                xq.put(t/time_unit)
                pvq.put(pv)
                cq.put(proc.c * 3600)

                co2q.put(co2_req)
                n2q.put(n2_req)
                o2q.put(o2_req)
                aq.put(air_req)
                baseq.put(base_req)

                co2ukq.put(co2_pid.Uk)
                co2upq.put(co2_pid.Up)
                co2uiq.put(co2_pid.Ui)
                co2udq.put(co2_pid.Ud)

                baseukq.put(base_pid.Uk)
                baseupq.put(base_pid.Up)
                baseuiq.put(base_pid.Ui)
                baseudq.put(base_pid.Ud)

        elif cmd == "SET_TIME":
            t = arg
        elif cmd == "UPDATE_VALUE":
            ob, name, value = arg
            rsp = _update_value(ops, state, proc, ob, name, value)
        elif cmd == "MODIFY_STATE":
            name, value = arg
            if name == "sp":
                sp = state['sp'] = value
            elif name == "pv":
                pv = state['pv'] = value
            elif name == "main_gas":
                mg = state['mg'] = value
            else:
                print("WARNING: Unrecognized name: %r" % name)
        else:
            raise ValueError(cmd)


In [212]:
def ph_sim(ops):
    # standalone sim function compatible with the do_sim_coroutine
    # function used for interactive use
    global state
    state = {}
    
    class MyList(list):
        put = list.append
    
    xq = MyList()
    pvq = MyList()
    cq = MyList()
    co2q = MyList()
    n2q = MyList()
    o2q = MyList()
    aq = MyList()
    baseq = MyList()
    co2ukq = MyList()
    co2upq = MyList()
    co2uiq = MyList()
    co2udq = MyList()
    baseukq = MyList()
    baseupq = MyList()
    baseuiq = MyList()
    baseudq = MyList()
    coro = ph_sim_coroutine(ops, state, xq, pvq, cq,
                     co2q, n2q, o2q, aq, baseq,
                     co2ukq, co2upq, co2uiq, co2udq,
                     baseukq, baseupq, baseuiq, baseudq)
    
    next(coro)
    coro.send(("SIM_ITERS", ops.end))
    
    data = list(zip(xq, pvq, cq,
                     co2q, n2q, o2q, aq, baseq))
    data2 = list(zip(co2ukq, co2upq, co2uiq, co2udq))
    data3 = list(zip(baseukq, baseupq, baseuiq, baseudq))
    return data, data2, data3
    

In [213]:
# O2 needs an extra padding space because the negative sign
# for N2 pgain doesn't count, i guesss
def fmt_float(f, decimals=3):
    if not f:
        return "0"
    f = str(f)
    if "." not in f:
        return f
    else:
        f = f.strip("0")
        if f[-1] == ".":
            f = f[:-1]
        if f[0] == ".":
            f = "0" + f
        if "." in f:
            if decimals > 0:
                f = f[:f.find(".")+decimals+1]
            else:
                f = f[:f.find(".")]
        return f
    
def get_text(ops):

    text = \
"""
CO2: P=%4d  I=%4d  Initial PV: %3d  k: %.4f/hr (%sx Est.)
                   Set Point:  %3d  c: %s%%/hr

 """ % (ops.co2_pid.p, ops.co2_pid.i,ops.initial_pv, ops.k*ops.k_mult, fmt_float(ops.k_mult),
        ops.set_point, fmt_float(ops.c))
    text = text.strip()
    return text

In [214]:
def run(ops):
    global data, data2, text, data3
    data, data2, data3 = ph_sim(ops)
    text = get_text(ops)
    plot(ops)

In [215]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, FormatStrFormatter
%matplotlib

def plot(ops):    
    print("Parsing Data")
    global x, pv, mg, cs, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a, uk, up, ui, ud
    x, pv, cs, co2_req, n2_req, o2_req, air_req, base_req = list(zip(*data))
    uk, up, ui, ud = list(zip(*data2))
    print("Closing Plot")
    plt.close()
    print("Plotting Data")

    step = 100
    xs = x[::step]
    xs = np.array(xs) / ops.plots.xscale_factor * ops.time_unit
    fig = plt.figure()
    ax1 = plt.subplot(311)
    ax2 = plt.subplot(312)
    ax3 = plt.subplot(313)

    pvs = np.array(pv[::step])
    ax1.plot(xs, pvs, "blue", ls="-", label="PV")
    ax1.axhline(y=sp, ls="--", color="black")
    # abs_err = np.abs(sp - pvs)
    # esq **= 2
    # esq = np.sqrt(esq)
    #ax1.plot(xs, abs_err, "red", ls="-", label="Err^2")

    #ax2.plot(xs, co2_req[::step], "purple", ls="-", label="CO2")
    ax2.plot(xs, n2_req[::step], "red", ls="-", label="N2")
    ax2.plot(xs, o2_req[::step], "green", ls="-", label="O2")
    ax2.plot(xs, air_req[::step], "cyan", ls="-", label="Air")
    ax2.plot(xs, co2_req[::step], "purple", ls="-", label="CO2")
    fm1 = FuncFormatter(lambda y, _: "%.2f"%y)
    fm2 = FuncFormatter(lambda y, _: "%.2f%%"%(y*100))

    ax3.plot(xs, uk[::step], "blue", ls="-", label="Uk")
    ax3.plot(xs, up[::step], "red", ls="-", label="Up")
    ax3.plot(xs, ui[::step], "green", ls="-", label="Ui")
    ax3.plot(xs, ud[::step], "purple", ls="-", label="Ud")

    ax1.yaxis.set_major_formatter(fm1)
    ax2.yaxis.set_major_formatter(fm2)
    ax2.set_ylim((0, 1.1))
    
    m, ma = ax1.get_ylim()
    nmin = np.min(pvs)
    nmax = np.max(pvs)
    m = nmin - 1
    ma = nmax + 1
    ax1.set_ylim((m, ma))


    for a in ax1, ax2, ax3: 
        b = a.get_position()
        a.set_position([b.x0, b.y0, b.width*0.9, b.height])
        a.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
        a.grid()
        if ops.plots.xscale == 'man':
            a.set_xlim((ops.plots.xmin, ops.plots.xmax))
    
    fig.text(0.15, 0.95, get_text(ops), transform=ax1.transAxes, fontsize=12,
        verticalalignment='top')
    
    wm=plt.get_current_fig_manager()
    wm.window.attributes('-topmost', 1)
    wm.window.attributes('-topmost', 0)
    # h = wm.window.winfo_height()
    # w = wm.window.winfo_width()
    wm.window.geometry("%sx%s+%s+%s"%(700,720,50, 20))

Using matplotlib backend: TkAgg


In [216]:
ops.end = 15 * hours
sp = 80

ops.co2_pid.p = -200
ops.co2_pid.i = 2
ops.co2_pid.amax = 25
ops.co2_pid.deadband = 0
ops.co2_pid.d = 0
ops.co2_pid.alpha = -1

ops.base_pid.p = 20
ops.base_pid.i = 2
ops.base_pid.amax = 100
ops.base_pid.deadband = 0
ops.base_pid.d = 0
ops.base_pid.alpha = -1

ops.initial_pv = 6
ops.set_point = 7.21
ops.c = 0
ops.d2c = 0.0

# k_mult = 1.25
ops.k_mult = 1
ops.k = pHProcess.default_k
ops.initial_actual_cno = (0,0,0)
ops.initial_request_cno = (0.00, 0, 0)
ops.mode = 'o2a'
ops.plots.xscale = 'auto'
ops.plots.xmin = 20
ops.plots.xmax = 30
ops.set_point_deadband = 0.02
ops.delay = 0
ops.bicarb = 3.7

run(ops)

from matplotlib.ticker import MultipleLocator
l = MultipleLocator(1)
for a in plt.gcf().axes:
    a.xaxis.set_major_locator(l)

Parsing Data
Closing Plot
Plotting Data


In [11]:
class RingBuffer():
    """ Numpy-based ring buffer """

    def __init__(self, maxsize, dtype=np.float32):
        if not maxsize or maxsize < 0:
            raise ValueError("%s requires max size argument" % self.__class__.__name__)
        self._queue = np.zeros(maxsize, dtype)
        self._dtype = dtype
        self._maxsize = maxsize
        self._end = 0
        self._sz = 0

    def put(self, d):
        self._queue[self._end] = d
        self._end += 1
        if self._end == self._maxsize:
            self._end = 0
        if self._sz < self._maxsize:
            self._sz += 1

    def put_list(self, lst):
        """
        :param lst: list to extend data from
        :type lst: list | tuple | np.ndarray
        """
        slen = len(lst)
        if slen > self._maxsize:
            self._queue = np.array(lst, dtype=self._dtype)[slen-self._maxsize:]
            self._sz = self._maxsize
            self._end = 0
            return
        if slen + self._end <= self._maxsize:
            start = self._end
            end = slen + self._end
            self._queue[start:end] = lst
            self._end = end
        else:
            # add slice in two steps
            first_step = self._maxsize - self._end
            self._queue[self._end: self._maxsize] = lst[:first_step]
            second_step = slen - first_step
            self._queue[:second_step] = lst[first_step:]
            self._end = second_step

        if self._sz < self._maxsize:
            self._sz += slen
        if self._end == self._maxsize:
            self._end = 0

    def extend(self, it):
        try:
            it.__len__  # list, tuple, nparray
        except AttributeError:
            it = tuple(it)
        self.put_list(it)

    def get(self):
        """
        Get method. Returns the entire queue but does NOT
        drain the queue.
        """
        if self._sz < self._maxsize:
            return self._queue[: self._end]
        else:
            return np.roll(self._queue, -self._end)

    def __len__(self):
        return self._sz

    def clear(self):
        self._end = 0
        self._sz = 0

In [12]:
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
from matplotlib.ticker import MultipleLocator, FuncFormatter, FormatStrFormatter
import tkinter as tk


class Data():
    def __init__(self, pts):
        self.values = RingBuffer(pts)
        self.pending = []
        self.put = self.pending.append
        
    def __len__(self):
        return len(self.values)
        
    def push(self, n):
        self.values.extend(self.pending[:n])
        self.pending = self.pending[n:]
        self.put = self.pending.append
        
    def get(self):
        return self.values.get()

    def clear(self):
        self.values.clear()
        self.pending.clear()
        
    def resize(self, new_sz):
        nv = RingBuffer(new_sz)
        self.values.extend(self.pending)
        self.pending = []
        self.put = self.pending.append
        nv.extend(self.values.get())
        self.values = nv

class SeriesData(Data):
    def __init__(self, name, ax, xdata, pts, **line_kw):
        self.name = name
        super().__init__(pts)
        self.x = xdata
        self.line, = ax.plot((), (), **line_kw)
        
    def update_line(self):
        self.line.set_data(self.x.get(), self.values.get())
        
    def hide(self):
        self.line.set_visible(False)
        
    def show(self):
        self.line.set_visible(False)
        
    def set_visible(self, visible):
        self.line.set_visible(visible)
        
    def get_visible(self):
        return self.line.get_visible()
    
    def toggle_visible(self):
        self.set_visible(not self.get_visible())
        
    
# Plot classes contain custom configuration code
# so SimWindow doesn't need to know how to set them up

class Plot():
    def __init__(self, ax, xdata, pts, yformatter=None, *series_ops):
        self.ax = ax
        self.x = xdata
        self.series = []
        for ops in series_ops:
            name, kw = ops
            # default
            kw['ls'] = kw.get('ls') or "-"
            s = SeriesData(name, ax, xdata, pts, **kw)
            self.series.append(s)
            setattr(self, name, s)
        b = ax.get_position()
        ax.set_position([b.x0, b.y0, b.width*0.88, b.height])
        ax.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
        self.leg_line_map = {}
        for lline, s in zip(ax.legend_.get_lines(), self.series):
            self.leg_line_map[lline] = s
        ax.grid()
        if yformatter:
            ax.yaxis.set_major_formatter(yformatter)
        self._hline = None
        
    def get_leg_map(self):
        return self.leg_line_map.copy()
            
    def put(self, *values):
        if len(values) != len(self.series):
            raise ValueError("Invalid argument to put: got %d values (expected %d)" \
                            % (len(values), len(self.series)))
        for s, v in zip(self.series, values):
            s.put(v)
            
    def push(self, n):
        for s in self.series:
            s.push(n)
            
    def update(self):
        for s in self.series:
            s.update_line()
        self.rescale()
        
    def rescale(self):
        self.ax.relim(True)
        self.ax.autoscale_view(True,True,True)
        
    def axhline(self, y, **kw):
        kw['ls'] = kw.get('ls') or "--"
        kw['color'] = kw.get('color') or 'black'
        if self._hline:
            self._hline.remove()
        self._hline = self.ax.axhline(y=y, **kw)
        
    def set_tick_interval(self, n):
        l = MultipleLocator(n)
        self.ax.xaxis.set_major_locator(l)
        
    def clear(self):
        for s in self.series:
            s.clear()
            

class PVPlot(Plot):
    """ Plot for PV """
    def __init__(self, ax, xdata, pts):
        series = [
            ("pv", dict(color="blue", ls="-", label="PV")),
            ("c", dict(color="green", ls="-", label="c"))
        ]
        fm = FormatStrFormatter("%.2f")
        super().__init__(ax, xdata, pts, fm, *series)
        self.c.line.set_visible(False)
        
    def rescale(self):
        self.ax.relim(True)
        self.ax.autoscale_view(True,True,True)
        lower, upper = self.ax.get_ybound()
        diff = upper - lower
        fpadding = 0.1
        spadding = fpadding * diff
        if spadding < 0.1:
            spadding = 0.1
        self.ax.set_ylim(lower - spadding, upper + spadding, True, None)
    
class GasesPlot(Plot):
    def __init__(self, ax, xdata, pts):
        
        series = [
            ("co2", dict(color="purple", ls="-", label="CO2")),
            ("n2", dict(color="red", ls="-", label="N2")),
            ("o2", dict(color="green", ls="-", label="O2")),
            ("air", dict(color="cyan", ls="-", label="Air")),
            ("base", dict(color="orange", ls="-", label="Base"))
        ]
        fm = FuncFormatter(lambda y, _: "%.2f%%"%(y*100))
        super().__init__(ax, xdata, pts, fm, *series)
        self.ax.set_ylim((-0.05, 1.1))
        
    def rescale(self):
        self.ax.relim(True)
        self.ax.autoscale_view(True,True,False)
        
        
class PIDPlot(Plot):
    def __init__(self, ax, xdata, pts):
        series = [
            ("uk", dict(color="blue", ls="-", label="Uk")),
            ("up", dict(color="red", ls="-", label="Up")),
            ("ui", dict(color="green", ls="-", label="Ui")),
            ("ud", dict(color="purple", ls="-", label="Ud")),
        ]
        
        super().__init__(ax, xdata, pts, None, *series)

In [13]:
class SimConfig(OptionCategory):
    simops = ops or SimOps()
    xwindow_hrs = 20
    update_interval = 100
    time_factor = 200
    time_unit = 3600
    x_tick_interval = 1         # spacing between ticks (in hours)
    
    fig_width  = 7  # inches
    fig_height = 9 # inches
cfg = SimConfig()

In [14]:
class EventDispatcher():
    def __init__(self):
        self.handlers = {}
        
    def create_event(self, ev):
        self.handlers[ev] = self.handlers.get(ev, [])
        
    def register(self, ev, handler):
        if ev not in self.handlers:
            self.create_event(ev)
        self.handlers[ev].append(handler)
        
    def unregister(self, ev, handler):
        self.handlers[ev].remove(handler)
        
    def fire(self, ev, *args):
        for h in self.handlers.get(ev, ()):
            h(ev, *args)
            
    def unregister_all(self, ev):
        self.handlers[ev] = []
        
    def create_callback(self, ev, *args):
        def cb():
            self.fire(ev, *args)
        return cb
            

In [15]:
class pHSimWindow():
    """ Sim Window handles common MPL functions & data storage 
    as well as non-MPL tkinter configuration 
    """
    def __init__(self, frame, sim_config, events):
        
        # Unpack config items
        self.cfg = sim_config
        self.update_interval = cfg.update_interval
        self.xwindow = cfg.xwindow_hrs
        
        # MPL objects
        self.fig = Figure(figsize=(self.cfg.fig_width, self.cfg.fig_height))
        ax1 = self.fig.add_subplot(411)
        ax2 = self.fig.add_subplot(412)
        ax3 = self.fig.add_subplot(413)
        ax4 = self.fig.add_subplot(414)
        
        self.xdata = Data(self.xwindow*3600)
        self.pv_plot = PVPlot(ax1, self.xdata, self.xwindow*3600)
        self.gases_plot = GasesPlot(ax2, self.xdata, self.xwindow*3600)
        self.cbase_pid_plot = PIDPlot(ax3, self.xdata, self.xwindow*3600)
        self.base_pid_plot = PIDPlot(ax4, self.xdata, self.xwindow*3600)
        
        x_spacing = self.cfg.time_unit / (self.cfg.x_tick_interval * 3600)
        self.pv_plot.set_tick_interval(x_spacing)
        self.gases_plot.set_tick_interval(x_spacing)
        self.cbase_pid_plot.set_tick_interval(x_spacing)
        self.base_pid_plot.set_tick_interval(x_spacing)

        # misc
        self.pv_plot.axhline(self.cfg.simops.set_point)
        
        # Tkinter setup
        self.root = frame
        
        self.fig_frame = tk.Frame(self.root)
        
        # a tk.DrawingArea
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.fig_frame)
        self.canvas.show()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

        self.toolbar = NavigationToolbar2TkAgg(self.canvas, self.root)
        self.toolbar.update()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        
        self._running = False
        self._monitor_id = None
        self._monitor_interval = 500
        
        # coroutine support
        self.coro = None
        self._current_state = {}
        
        # Periodic update support
        self.updating = True
        
        # Event handling for callbacks
        # All event handling works by modifying
        # Commands are sent directly to the coroutine
        # via use of command codes. 
        # The primary code is (obviously) used to run
        # the simulation. All the codes used below 
        # signal the coroutine to take specific action,
        # including modifying stateful values
        
        self.events = events
        
        def register_update_state(event, arg):
            def on_modified(ev, value, *args):
                self.trigger_update_state(arg, value)
                print("State Value Updated: %s %s " % (arg, value))
            self.events.register(event, on_modified)
                
        register_update_state("SP_CHANGED", "sp")
        register_update_state("PV_CHANGED", "pv")
        register_update_state("MG_CHANGED", "mg")
        
        def set_target_line(event, value):
            self.pv_plot.axhline(value)
        self.events.register("SP_CHANGED", set_target_line)

        
        def on_pid_changed(ev, value, *args):
            ev = ev.lower()
            ctrl, param, _ = ev.split("_")
            ctrl += "_pid"
            print("Value Changed: %s.%s: %.2f" % (ctrl, param, value))
            self.trigger_update_value(ctrl, param, value)
            
        for c in "CO2", "BASE":
            for p in "PGAIN", "ITIME", "DTIME", "BETA", "MODE", "MAN":
                ev = "%s_%s_CHANGED" % (c,p)
                self.events.register(ev, on_pid_changed)
        
        def on_process_gain_changed(ev, value, *args):
            c = ev.split("_")[0].lower()
            if c == "kmult":
                c = "k_mult"
            self.trigger_update_value("process", c, value)
            print("Changed value: %s to %s" % (c, value))
            
        for c in "K", "KMULT", "C", "DC", "D2C", "DELAY":
            ev = c + "_CHANGED"
            self.events.register(ev, on_process_gain_changed)
            
        def on_timefactor_changed(ev, value, *args):
            self.cfg.time_factor = value
            print(ev, ":", value)
        self.events.register("TIMEFACTOR_CHANGED", on_timefactor_changed)
        
        def on_clear_all_data(ev, *args):
            self.xdata.clear()
            for p in self.iter_plots():
                p.clear()
            self.coro.send(("SET_TIME", 0))
        self.events.register("CLEAR_ALL_DATA", on_clear_all_data)
        
        def on_advance_sim(ev, value, *args):
            if not value: return
            try:
                value = int(value)
            except ValueError:
                print("Invalid value: %r" % value)
                return
            print("Advancing simulation %d seconds..." % value)
            self._run_sim(value)
        self.events.register("ADVANCE_SIM", on_advance_sim)
        
        def on_legend_clicked(event):
            ll = event.artist
            series = self.leg_map[ll]
            series.toggle_visible()
            
        def on_xwindow_changed(ev, value, *args):
            nv = int(value * 3600)
            for p in self.iter_plots():
                for s in p.series:
                    if len(s) != nv:
                        s.resize(nv)
            if len(self.xdata) != nv:
                self.xdata.resize(nv)
        self.events.register("XWINDOW_CHANGED", on_xwindow_changed)
            
        # legend picker support
        self.leg_map = {}
        for p in self.iter_plots():
            m = p.get_leg_map()
            for leg, s in m.items():
                leg.set_picker(5)
                self.leg_map[leg] = s
        self.fig.canvas.mpl_connect("pick_event", on_legend_clicked)

    def trigger_update_value(self, ob, name, value):
        cmd = "UPDATE_VALUE"
        args = (ob, name, value)
        self.coro.send((cmd, args))
        
    def trigger_update_state(self, name, value):
        cmd = "MODIFY_STATE"
        args = (name, value)
        self.coro.send((cmd, args))
        
    def iter_plots(self):
        return (self.pv_plot, self.gases_plot, self.cbase_pid_plot, self.base_pid_plot)
        
    def pack(self, side=tk.TOP, fill=tk.BOTH, expand=1):
        self.fig_frame.pack(side=side, fill=fill, expand=expand)
        
    def start(self):
        self._updating = True
        self.begin_sim()
        self.schedule_update()
        
    def stop(self):
        self._updating = False
        
    def resume(self):
        if not self.coro:
            self.start()
        else:
            self.schedule_update()
        
    def begin_sim(self):
        self.coro = ph_sim_coroutine(self.cfg.simops, self._current_state,
                         self.xdata, self.pv_plot.pv, self.pv_plot.c, 
                         self.gases_plot.co2, self.gases_plot.n2, self.gases_plot.o2, 
                         self.gases_plot.air, self.gases_plot.base,
                         self.cbase_pid_plot.uk, self.cbase_pid_plot.up, self.cbase_pid_plot.ui, self.cbase_pid_plot.ud,
                         self.base_pid_plot.uk, self.base_pid_plot.up, self.base_pid_plot.ui, self.base_pid_plot.ud)
        next(self.coro)
        
    def update(self):
        n = self.cfg.time_factor
        n = int(n)
        self._run_sim(n)
        self.events.fire("PROCESS_STATE_UPDATE", self._current_state.copy())
        
    def _run_sim(self, n):
        self.coro.send(("SIM_ITERS", n))
        
        self.xdata.push(n)
        self.pv_plot.push(n)
        self.gases_plot.push(n)
        self.base_pid_plot.push(n)
        self.cbase_pid_plot.push(n)
        
        self.pv_plot.update()
        self.gases_plot.update()
        self.cbase_pid_plot.update()
        self.base_pid_plot.update()
        
        self.fig.canvas.draw()
        
    def schedule_update(self):
        def update(self=self):
            if self.updating:
                self.update()
                self.root.after(self.update_interval, update)
        self.root.after(self.update_interval, update)    
    


In [16]:
class TkQuitHack(tk.Tk):
    def _hacky_monitor_running(self):
        # hack to allow closing tkinter window by 
        # "X" button in this interactive prompt
        # unsure if this is an ipython/mpl artifact or not
        # but it works
        def monitor():
            if not self._hacky_running:
                for cb in self._hacky_on_destroyed:
                    cb()
                self.quit()
                self.destroy()
                self._hacky_destroyed = True
                
            else:
                self.after(self._hacky_monitor_interval, monitor)
        self.after(self._hacky_monitor_interval, monitor)
    def __init__(self, *args, **kw):
        super().__init__(*args, **kw)
        self._hacky_running = True
        self._hacky_monitor_interval = 200
        self._hacky_monitor_running()
        self.protocol("WM_DELETE_WINDOW", self.hacky_destroy)
        self._hacky_destroyed = False
        self._hacky_on_destroyed = []
        
    def hacky_destroy(self):
        self._hacky_running = False
        
    def register_destroy_callback(self, cb):
        self._hacky_on_destroyed.append(cb)
        
    def is_destroyed(self):
        return self._hacky_destroyed

In [17]:
import tkinter as tk, tkinter.ttk as ttk
class LabeledEntry(tk.Frame):
    def __init__(self, master, text, def_value, events, value_changed_event, pos="top", *args, **kw):
        super().__init__(master, *args, **kw)
        self.label = tk.Label(self, text=text)
        self.entry = tk.Entry(self)
        
        if pos.lower() == "top":
            r1, c1 = 0, 0
            r2, c2 = 1, 0
        elif pos.lower() == "left":
            r1, c1 = 0, 0
            r2, c2 = 0, 1
        elif pos.lower() == "right":
            r1, c1 = 0, 1
            r2, c2 = 0, 0
        else:
            raise ValueError("Bad Position: %r" % pos)
            
        self.label.grid(row=r1, column=c1, sticky=(tk.E, tk.W))
        self.entry.grid(row=r2, column=c2, sticky=(tk.E, tk.W))
            
        if def_value is None:
            def_value = ""
        self.value = def_value
        self.set(def_value)
        self._trigger_mode = "on_change"
            
        self.entry.bind("<Return>", self.maybe_value_changed)
        self.entry.bind("<FocusOut>", self.maybe_value_changed)
        self.events = events
        self.value_changed_event = value_changed_event
        
    def label_width(self, value):
        self.label.config(width=value)
        
    def entry_width(self, value):
        self.entry.config(width=value)
        
    def maybe_value_changed(self, _):
        value = self.get()
            
        try:
            v = float(value or 0)
        except ValueError:
            print("Could not coerce string to float: %r" % value)
            return
        
        if v == float(self.value) and self._trigger_mode == "on_change":
            return
        
        self.value = value
        self.trigger_value_changed(v)
        
    def trigger_value_changed(self, v):
            self.events.fire(self.value_changed_event, v)
        
    def get(self):
        return self.entry.get()
    
    def set(self, value):
        value = str(value)
        if not value:
            value = "0"
        self.entry.delete(0, tk.END)
        self.entry.insert(0, value)
        self.value = value
        
    def set_trigger_mode(self, mode):
        if mode not in ("always", "on_change"):
            raise ValueError("Unrecognized mode: %r" % mode)
        self._trigger_mode = mode
        
        
class EntryButton(tk.Frame):
    def __init__(self, master, text, events, event):
        super().__init__(master)
        self.entry = tk.Entry(self)
        self.entry.insert(0, 3600)
        self.button = ttk.Button(self, text=text, command=self.fire)
        self.entry.bind("<Return>", self.fire)
        self.events = events
        self.event = event
        
        self.entry.grid(row=0, column=1)
        self.button.grid(row=1, column=1)
        
    def fire(self, e=None):
        self.events.fire(self.event, self.entry.get())
    

# Patched tkinter settit. For some reason, setting the internal
# Variable isn't properly being used to update the OptionMenu
# widget text. 
      
# Also, this makes more sense as a function constructor
# than a class (imo)
    
def _setit(menu, var, value, cb=None):
    def func(*args):
        var.set(value)
        menu.config(text=value)
        if cb is not None:
            cb(value, *args)
    return func
        
# Patched tkinter optionmenu to work properly. The only changes
# are changing the default "textvariable" to "text" in kw to super,
# and re-running the code to bind to the patched _settit class. 

class PatchedOptionMenu(tk.Menubutton):
    """OptionMenu which allows the user to select a value from a menu."""
    def __init__(self, master, variable, value, *values, **kwargs):
        """Construct an optionmenu widget with the parent MASTER, with
        the resource textvariable set to VARIABLE, the initially selected
        value VALUE, the other menu values VALUES and an additional
        keyword argument command."""
        kw = {"borderwidth": 2, "text": value,
              "indicatoron": 1, "relief": tk.RAISED, "anchor": "c",
              "highlightthickness": 2}
        tk.Widget.__init__(self, master, "menubutton", kw)
        self.widgetName = 'tk_optionMenu'
        menu = self.__menu = tk.Menu(self, name="menu", tearoff=0)
        self.menuname = menu._w
        # 'command' is the only supported keyword
        callback = kwargs.get('command')
        if 'command' in kwargs:
            del kwargs['command']
        if kwargs:
            raise TclError('unknown option -'+kwargs.keys()[0])
        menu.add_command(label=value,
                 command=_setit(self, variable, value, callback))
        for v in values:
            menu.add_command(label=v,
                     command=_setit(self, variable, v, callback))
        self["menu"] = menu

    def __getitem__(self, name):
        if name == 'menu':
            return self.__menu
        return tk.Widget.__getitem__(self, name)

    def destroy(self):
        """Destroy this widget and the associated menu."""
        tk.Menubutton.destroy(self)
        self.__menu = None
        
class MenuFrame(tk.LabelFrame):
    def __init__(self, master, text, events, event, entries):
        super().__init__(master, text=text)
        self.entries = entries
        self.events = events
        self.event = event
        
        self.entries = []
        entry_names = []
        
        for i, (var, val, ev) in enumerate(entries, 1):
            w = LabeledEntry(self, var, val, events, ev)
            w.grid(row=i, column=0)
            self.entries.append(w)
            entry_names.append(var)
        
        self.tkvar = tk.StringVar(self)
        self.tkvar.set(entry_names[0])
        self.menu = OptionMenu(self, self.tkvar, *entry_names)
        self.menu.grid(column=0, row=0)
        self.events.register(self.event, self.on_menu_changed)
            
    def on_menu_changed(self, _):
        self.events.fire(self.event, self.tkvar.get())
        
            
class LabelFrameWithLabelEntries(tk.LabelFrame):
    def __init__(self, master, text, events, entries=()):
        super().__init__(master, text=text)
        self.events = events
        self.entries = []
        for text, val, ev in entries:
            self.add_entry(text, val, ev)
            
    def set_trigger_mode(self, mode):
        for e in self.entries:
            e.set_trigger_mode(mode)
            
    def add_entry(self, text, val, ev):
        e = LabeledEntry(self, text, val, self.events, ev, pos="left")
        e.label_width(10)
        e.entry_width(8)
        e.grid(row=len(self.entries), column=0)
        self.entries.append(e)
            
class LabelDisplay(tk.Frame):
    def __init__(self, master, events, event, text, value="", w1=None, w2=None):
        super().__init__(master)
        self.label = tk.Label(self, text=text, anchor=tk.E)
        self.display = tk.Label(self, text=value, anchor=tk.E)
        if w1:
            self.label.config(width=w1)
        if w2:
            self.display.config(width=w2)
        
        self.label.grid(row=0, column=0, sticky=(tk.W, tk.E))
        self.display.grid(row=0, column=1, sticky=(tk.W, tk.E))
        self.events = events
        self.events.register(event, self.on_value_update)
        
    def on_value_update(self, ev, value, *args):
        self.set(value)
        
    def set(self, value):
        value = str(value)
        self.display['text'] = value
    
    def get(self):
        return self.display['text']
        
class LabelDisplayFrame(tk.LabelFrame):
    def __init__(self, master, text, events):
        super().__init__(master, text=text)
        self.labels = {}
        self.events = events
        
    def add_label(self, id, event, text, value="", w1=None, w2=None):
        label = LabelDisplay(self, self.events, event, text, value, w1, w2)
        label.grid(row=len(self.labels), column=0, sticky=(tk.E, tk.W, tk.N, tk.S))
        self.labels[id] = label
        
    def add_labels(self, *args):
        for a in args:
            self.add_label(*args)
        
class Combobox(ttk.Combobox):
    def __init__(self, master, **kw):
        super().__init__(master, **kw)
        vcmd = self.register(self.nomodify,)
        self.config(validate="key", validatecommand=vcmd)
        self.bind("<Return>", lambda _: self.event_generate("<1>"))
        self.bind("<Button-1>", lambda _: self.event_generate("<Down>", when='head'))
        self.current(0)
        
    def nomodify(self):
        return False
        
class MenuLabel(tk.Frame):
    def __init__(self, master, text, events, event, values, w1=10, w2=6, map=None, **kw):
        super().__init__(master, **kw)
        self.label = tk.Label(self, text=text)
        self.label.config(width=w1)
        
        if map is None:
            map = {k:k for k in values}
        else:
            if not all(k in map for k in values):
                raise ValueError("Unmapped keys: " % " ".join(set(values)-set(map)))
        
        self.map = map
        self.events = events
        self.event = event
        
        self.menu = Combobox(self, values=values)
        self.menu.bind("<<ComboboxSelected>>", self.menu_selected)
        self.menu.config(width=w2)
        
        self.label.grid(row=0, column=0, sticky=(tk.E, tk.W, tk.N, tk.S))
        self.menu.grid(row=0, column=1, sticky=(tk.E, tk.W, tk.N, tk.S))
        
        
    def get_raw(self):
        return self.menu.get()
    
    def get(self):
        return self.map[self.menu.get()]
        
    def menu_selected(self, e):
        self.events.fire(self.event, self.get())
        
    

In [26]:
class StatusDisplay(LabelDisplayFrame):
    def __init__(self, master, text, events, cfg):
        super().__init__(master, text, events)

        # text config....
        c,n,o = cfg.simops.initial_request_cno
        b = cfg.simops.initial_request_base
        a = 1-(c+n+o)
        sub2 = "\u2082:"
        co2 = "CO"+sub2
        n2 = "N"+sub2
        o2 = "O"+sub2
        
        self.add_label("PV", "PV_PROCESS_UPDATE", "PV:", "%.2f"%(cfg.simops.initial_pv), 5, 6)
        self.add_label("CO2", "CO2_PROCESS_UPDATE", co2, "%.1f%%" % (c*100), 5, 6)
        self.add_label("N2", "N2_PROCESS_UPDATE", n2, "%.1f%%" % (n*100), 5, 6)
        self.add_label("O2", "O2_PROCESS_UPDATE", o2, "%.1f%%" % (o*100), 5, 6)
        self.add_label("Air", "AIR_PROCESS_UPDATE", "Air:", "%.1f%%" % (a*100), 5, 6)
        self.add_label("Base", "BASE_PROCESS_UPDATE", "Base:", "%.1f%%" % b, 5, 6)
        
        self.add_label("c", "C_PROCESS_UPDATE", "c:", cfg.simops.c, 3, 6)
        self.add_label("dc", "DC_PROCESS_UPDATE", "dc:", cfg.simops.dc, 3, 6)
        self.add_label("d2c", "D2C_PROCESS_UPDATE", "d\u00B2c:", cfg.simops.d2c, 3, 6)
        
        self.events.register("PROCESS_STATE_UPDATE", self.on_state_update)
        
    def on_state_update(self, ev, state, *args):
        self.labels['c'].set("%.4g" % state['c'])
        self.labels['dc'].set("%.4g" % state['dc'])
        self.labels['d2c'].set("%.4g" % state['d2c'])
        self.labels['PV'].set("%.2f" % state['pv'])
        
        self.labels["CO2"].set("%.1f%%" % (state['co2_req']*100))
        self.labels["N2"].set("%.1f%%" % (state['n2_req']*100))
        self.labels["O2"].set("%.1f%%" % (state['o2_req']*100))
        self.labels["Air"].set("%.1f%%" % (state['air_req']*100))
        self.labels["Base"].set("%.1f%%" % (state['base_req']*100))
        
        
class PIDFrame(LabelFrameWithLabelEntries):
    def __init__(self, master, text, events, prefix, pid_ops):
        super().__init__(master, text, events)
        
        self.add_entry("PGain:", pid_ops.p, prefix + "_PGAIN_CHANGED")
        self.add_entry("ITime:", pid_ops.i, prefix + "_ITIME_CHANGED")
        self.add_entry("DTime:", pid_ops.d, prefix + "_DTIME_CHANGED")
        self.add_entry("Beta:", pid_ops.beta, prefix + "_BETA_CHANGED")
        self.add_entry("Man:", pid_ops.man_request, prefix + "_MAN_CHANGED")
        
        # Modes
        modes = [
            ("Auto", PIDController.AUTO),
            ("Man", PIDController.MAN),
            ("Off", PIDController.OFF)
        ]
        mode_order = [m[0] for m in modes]
        map = dict(modes)
        self.menu = MenuLabel(self, "Mode:", events, prefix + "_MODE_CHANGED", mode_order, map=map)
        self.menu.grid(row=len(self.entries), column=0, sticky=(tk.E,tk.W,tk.N,tk.S))
              

In [27]:

class PIDSimFrame():
    def __init__(self, master, WindowClass, sim_cfg, events=None):
        if events is None:
            events = EventDispatcher()
        self.cfg = sim_cfg
        ops = cfg.simops
        self.events = events
        self.frame = tk.Frame(master)
        self.w = WindowClass(self.frame, sim_cfg, events)
        self.w.pack(side=tk.LEFT, fill=tk.BOTH, expand=1)
        self.settings_frame = sf = tk.Frame(self.frame)
        self.settings_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=1)
        
        # Widgets
        
        # CO2 PID
        co2_pid = PIDFrame(sf, "CO\u2082 PID:", events, "CO2", ops.co2_pid)
        
        # Base PID
        base_pid = PIDFrame(sf, "Base PID:", events, "Base", ops.base_pid)
        
        # Process values
        proc = LabelFrameWithLabelEntries(sf, "Process:", events)
        proc.add_entry("Set Point:", ops.set_point, "SP_CHANGED")
        proc.add_entry("PV:", ops.initial_pv, "PV_CHANGED")
        proc.add_entry("k:", ops.k, "K_CHANGED")
        proc.add_entry("k_mult:", ops.k_mult, "KMULT_CHANGED")
        proc.add_entry("delay:", s2m(ops.delay), "DELAY_CHANGED")
        
        # Simulation options
        sim = LabelFrameWithLabelEntries(sf, "Simulation:", events)
        sim.add_entry("time factor:", cfg.time_factor, "TIMEFACTOR_CHANGED")
        sim.add_entry("x window(hr):", cfg.xwindow_hrs, "XWINDOW_CHANGED")
        
        clear = ttk.Button(sf, text="Clear", command=events.create_callback("CLEAR_ALL_DATA"))
        advance = EntryButton(sf, "Advance(s)", events, "ADVANCE_SIM")

        # Consumption constants
        c_frame = LabelFrameWithLabelEntries(sf, "Consumption:", events)
        c_frame.add_entry("c", ops.c, "C_CHANGED")
        c_frame.add_entry("dc/dt", 0, "DC_CHANGED")
        c_frame.add_entry("d\u00B2c/dt\u00B2", 0, "D2C_CHANGED")
        c_frame.set_trigger_mode("always")
        
        # Output display
        cstate = StatusDisplay(sf, "Current:", events, self.cfg)        

        tk_all = (tk.E, tk.W, tk.N, tk.S)
        co2_pid.grid(row=1, column=1, sticky=tk_all)
        base_pid.grid(row=1, column=2, sticky=tk_all)
        
        proc.grid(row=2, column=1, sticky=tk_all)
        sim.grid(row=2, column=2, sticky=tk_all)

        c_frame.grid(row=3, column=1, rowspan=3, sticky=(tk.E, tk.W, tk.N, tk.S))
        cstate.grid(row=3, column=2, rowspan=8, sticky=(tk.E, tk.W, tk.N, tk.S))
        
        advance.grid(row=11, column=1)
        clear.grid(row=11, column=2, sticky=tk.S)
        
        self.wl = {}
        self.wl.update((k,v) for k,v in locals().items() if isinstance(v, tk.Widget))
        
        self.w.start()
        master.register_destroy_callback(self.w.stop)
        
        self.co2_pid = co2_pid
        self.base_pid = base_pid
        
    def pack(self, **kw):
        self.frame.pack(**kw)
        
        

In [217]:
def run_sim():
    cfg.simops.co2_pid.p = -200
    cfg.simops.co2_pid.i = 60
    cfg.simops.co2_pid.amax = 30
    cfg.simops.co2_pid.deadband = 0

    cfg.simops.base_pid.p = 20
    cfg.simops.base_pid.i = 2
    cfg.simops.base_pid.amax = 100
    cfg.simops.base_pid.deadband = 0

    cfg.simops.main_gas = 1.0

    cfg.simops.set_point_deadband = 0.02

    cfg.update_interval = 50
    cfg.time_factor = 200

    cfg.simops.initial_pv = 7
    cfg.simops.set_point = 7.21
    cfg.xwindow_hrs = 5
    cfg.simops.k = pHProcess.default_k

    root = TkQuitHack()
    root.wm_title("pH Simulation")
    #root.geometry("%dx%d"%(600, 800))
    f = PIDSimFrame(root, pHSimWindow, cfg)
    f.pack()
    root.mainloop()